In [69]:
from sklearn.datasets import fetch_20newsgroups
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from collections import Counter
import math
from google.colab import drive
drive.mount('/content/drive')
from sklearn.datasets import load_files
import pickle
import numpy as np
import matplotlib.pyplot as plt
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [21]:
url = '/content/drive/MyDrive/USML HW2/newsgrps_test.pkl'
url2 = '/content/drive/MyDrive/USML HW2/newsgrps_test_no_footers_headers_quotes.pkl'

In [22]:
with open(url2, 'rb') as file:
  ng_test = pickle.load(file)

In [23]:
len(ng_test.data)

7532

In [24]:
len(ng_test.target)

7532

In [13]:
i = 10
print(ng_test.data[i],ng_test.target_names[ng_test.target[i]])

From: Greg.Reinacker@FtCollins.NCR.COM
Subject: Windows On-Line Review uploaded
Reply-To: Greg.Reinacker@FtCollinsCO.NCR.COM
Organization: NCR Microelectronics, Ft. Collins, CO
Lines: 12

I have uploaded the Windows On-Line Review shareware edition to
ftp.cica.indiana.edu as /pub/pc/win3/uploads/wolrs7.zip.

It is an on-line magazine which contains reviews of some shareware
products...I grabbed it from the Windows On-Line BBS.

--
--------------------------------------------------------------------------
Greg Reinacker                          (303) 223-5100 x9289
NCR Microelectronic Products Division   VoicePlus 464-9289
2001 Danfield Court                     Greg.Reinacker@FtCollinsCO.NCR.COM
Fort Collins, CO  80525
 comp.os.ms-windows.misc


In [28]:
i = 10
print(ng_test.data[i])
print(ng_test.target_names[ng_test.target[i]])

I have uploaded the Windows On-Line Review shareware edition to
ftp.cica.indiana.edu as /pub/pc/win3/uploads/wolrs7.zip.

It is an on-line magazine which contains reviews of some shareware
products...I grabbed it from the Windows On-Line BBS.

--
comp.os.ms-windows.misc


In [35]:
total_text = ''
for i in range(len(ng_test.data)):
  total_text += ng_test.data[i]

In [48]:
text = ng_test.data[0]

In [55]:
len(total_text)

8261569

In [56]:
from nltk.corpus import wordnet

def is_valid_word(word):
  return bool(wordnet.synsets(word))

total_text = re.sub(r'[^a-zA-Z0-9 \n]', '', total_text)
total_text = re.sub(r'\n+', ' ', total_text)
total_text = total_text.lower()
tokens = word_tokenize(total_text)
stop_words = set(stopwords.words('english'))
tokens = [word for word in tokens if word not in stop_words]
filtered_tokens = [word for word in tokens if is_valid_word(word)]

In [57]:
len(filtered_tokens)

601285

In [65]:
filtered_tokens = set(filtered_tokens)

In [66]:
len(filtered_tokens)

28892

In [70]:
filtered_tokens_5k = random.sample(filtered_tokens, 5000)

<ipython-input-70-ffbac7204284>:1: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  filtered_tokens_5k = random.sample(filtered_tokens, 5000)


In [74]:
filtered_tokens_5k.sort()

In [81]:
def preprocess_text(text):
  text = re.sub(r'[^a-zA-Z0-9 \n]', '', text)
  text = re.sub(r'\n+', ' ', text)
  text = text.lower()
  tokens = word_tokenize(text)
  stop_words = set(stopwords.words('english'))
  tokens = [word for word in tokens if word not in stop_words]
  dict_tokens = dict(Counter(tokens))

  np_arr = np.zeros(5000)
  for i in range(len(filtered_tokens_5k)):
    if filtered_tokens_5k[i] in dict_tokens:
      np_arr[i] = dict_tokens[filtered_tokens_5k[i]]
  return np_arr

In [99]:
np_dataset = []

for i in range(len(ng_test.data)):
  np_dataset.append(preprocess_text(ng_test.data[i]))

In [131]:
idx = 1999
for i in range(len(np_dataset[idx])):
  if np_dataset[idx][i] != 0:
    print(filtered_tokens_5k[i],np_dataset[idx][i])

21 1.0
40 1.0
connector 1.0
happily 1.0
jitters 1.0
macintosh 1.0
plus 1.0
subject 1.0
work 1.0


In [106]:
dataset = np.array(np_dataset)

In [107]:
dataset.shape

(7532, 5000)

In [108]:
dataset.nbytes

301280000

In [109]:
zero_count = np.count_nonzero(dataset == 0)

# Calculate the total number of elements in the matrix
total_elements = dataset.size

# Calculate the sparsity (percentage of zero elements)
sparsity = (zero_count / total_elements) * 100

print(f"The sparsity of the dataset is {sparsity:.2f}%")

The sparsity of the dataset is 99.82%


In [132]:
filename = '/content/drive/MyDrive/USML HW2/dataset.pkl'

with open(filename, 'wb') as file:
  pickle.dump(dataset, file)
